In [5]:
from mpmath import mp, mpf, findroot, j0, besselj
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

def format_dyadic_hex(value):
    l = hex(value)[2:]
    n = 8
    x = [l[i:i + n] for i in range(0, len(l), n)]
    return "0x" + "_".join(x) + "_u128"

def print_dyadic(value):
    (s, m, e) = RealField(128)(value).sign_mantissa_exponent();
    print("DyadicFloat128 {")
    print(f"    sign: DyadicSign::{'Pos' if s >= 0 else 'Neg'},")
    print(f"    exponent: {e},")
    print(f"    mantissa: {format_dyadic_hex(m)},")
    print("},")

In [6]:
# searching for zeros and extremas
R120 = RealField(120)

zeros = []

mp.prec = 200

step = mpf("0.01")
epsilon = mpf("1e-35")
x = mpf("0.0")

def j0_prime(x):
    return diff(lambda t: besselj(0, t), x)

previous_zero = R120(0)
j0_zeros = []

while x < mpf("76.0"):
    f1 = besselj(0, x)
    f2 = besselj(0, x + step)
    if f1 * f2 < 0:
        zero = findroot(lambda t: j0(t), (x, x + step), solver='bisect', tol=mp.mpf("1e-41"))
        previous_zero = zero
        j0_zeros.append(zero)
    if previous_zero is not None and abs(x - mpf(f'{round(x)}')) < epsilon:
        zeros.append(previous_zero)
    x += step

j0_extrema = []

x = mpf("0.0")
while x < mpf("76.0"):
    d1 = mp.diff(lambda t: j0(t), x)
    d2 = mp.diff(lambda t: j0(t), x + step)
    if d1 * d2 < 0:
        extremum = findroot(lambda t: mp.diff(lambda u: j0(u), t), (x, x + step), solver='bisect', tol=mp.mpf("1e-41"))
        j0_extrema.append(extremum)
    x += step

# Print results
for i, z in enumerate(j0_zeros):
    print(f"Zero {i+1}: x ≈ {z}")

print("Extrema (peaks/valleys) of J0(x):")
for e in j0_extrema:
    print(f"nExtrema: {e}")

j0_zeros.extend(j0_extrema)

j0_zeros = sorted(j0_zeros)

# Print results
for i, z in enumerate(j0_zeros):
    print(f"Peak or zero {i+1}: x ≈ {z}")

print("")

# print("pub(crate) static J0_ZEROS: [(u64, u64); 48] = [")
# print(f"(0x0, 0x0),")
# for z in j0_zeros:
#     k = split_double_double(z)
#     hi = double_to_hex(k[1])
#     lo = double_to_hex(k[0])
#     print(f"({lo}, {hi}),")
    
# print("];")

# print("pub(crate) static J0_ZEROS_RATIONAL128: [DyadicFloat128; 48] = [")
# print(f"DyadicFloat128 {{ sign: DyadicSign::Pos, exponent: 0, mantissa: 0x0u128, }},")
# for z in j0_zeros:
#     print_dyadic(z)
    
# print("];")

Zero 1: x ≈ 2.4048255576957727686216318793264573074939733531189268383204
Zero 2: x ≈ 5.5200781102863106495966041128130242000048191809601652765466
Zero 3: x ≈ 8.6537279129110122169541987126609444591156073271620997480225
Zero 4: x ≈ 11.791534439014281613743044911925461814515010486679185542023
Zero 5: x ≈ 14.930917708487785947762593997388678583649734231732812761062
Zero 6: x ≈ 18.071063967910922543147882975618174740290230790039575425635
Zero 7: x ≈ 21.211636629879258959078393350526310414504749137725282245072
Zero 8: x ≈ 24.352471530749302737057944763178904145854465408084454673987
Zero 9: x ≈ 27.493479132040254795877288234607417325353844361837018425873
Zero 10: x ≈ 30.634606468431975117549578926854233032528273435206877073822
Zero 11: x ≈ 33.775820213573568684238546346714724828060649562571856542741
Zero 12: x ≈ 36.917098353664043979769493063272951049055170549696852462216
Zero 13: x ≈ 40.058425764628239294799307373994471729106099867476853029585
Zero 14: x ≈ 43.1997917131767303575240727287434

In [7]:
def compute_intervals(zeros):
    intervals = []
    for i in range(0, len(zeros)):
        if i == 0:
            a = (zeros[i]) / 2 - 0.05 - zeros[i]
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.05 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        elif i + 1 > len(zeros) - 1:
            a = (zeros[i - 1] + zeros[i]) / 2 - 0.05 - zeros[i]
            b = (zeros[i]) + 0.83 + 0.05 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        else:
            a = (zeros[i - 1] + zeros[i]) / 2 - zeros[i] - 0.05
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.05  - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
    return intervals

intervals = compute_intervals(j0_zeros)
# print(intervals)

def build_sollya_script(a, b, zero, deg):
    return f"""
prec = 500;
bessel_j0 = library("/Users/radzivon/RustroverProjects/pxfm/notes/bessel_sollya/cmake-build-release/libbessel_sollya.dylib");
f = bessel_j0(x + {zero});
d = [{a}, {b}];
pf = remez(f, {deg}, d);
for i from 0 to degree(pf) do {{
    write(coeff(pf, i)) >> "coefficients.txt";
    write("\\n") >> "coefficients.txt";
}};
"""

def load_coefficients(filename):
    with open(filename, "r") as f:
        return [RR(line.strip()) for line in f if line.strip()]

def call_sollya_on_interval(a, b, zero, degree=12):
    sollya_script = build_sollya_script(a, b, zero, degree)
    with open("tmp_interval.sollya", "w") as f:
        f.write(sollya_script)
    import subprocess
    if os.path.exists("coefficients.txt"):
        os.remove("coefficients.txt")
    try:
        result = subprocess.run(
            ["sollya", "tmp_interval.sollya"],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        return

degree = 13

print(f"pub(crate) static J0F_COEFFS: [[u64;{degree + 1}]; {len(intervals)}] = [")
for i in range(0, len(intervals)):
    interval = intervals[i]
    call_sollya_on_interval(interval[0], interval[1], interval[2], degree)
    coeffs = load_coefficients(f"coefficients.txt")
    print("[")
    for c in coeffs:
        print(double_to_hex(c) + ",")
    print("],")
print("];")

pub(crate) static J0F_COEFFS: [[u64;14]; 47] = [
[
0xbca69de51bcc0120,
0xbfe09cdb3655127d,
0x3fbba1deea029925,
0x3facfae8643687e7,
0xbf81bb1cbe1caeb6,
0xbf61f992590d3f89,
0x3f315382bbf1a06b,
0x3f06ed3ba591e6da,
0xbed232c8d8aba5cc,
0xbea1cceb52db45ee,
0x3e68006b6b5682ca,
0x3e329cc1da0793f0,
0xbdf5dbb0053da32b,
0xbdc04e14e5d96539,
],
[
0xbfd9c6cf582cbf7f,
0x3cb1d608597b5d8f,
0x3fc9c6cf582cbf55,
0xbf91f06d14e12b29,
0xbf8b589d1da0f7e4,
0x3f50f9103d00ae93,
0x3f38644561ce9290,
0xbefa2a034c749192,
0xbed83a0686fd1dd2,
0x3e96a5085d7844b5,
0x3e6ebfcaacd49b87,
0xbe2964b9932b57f3,
0xbdfad73cf4701511,
0x3db5acbea609d3e5,
],
[
0xbc6a1105f9dbcdd5,
0x3fd5c6e60a097826,
0xbf9f8f72e7a8471e,
0xbfab2150cb41ece4,
0x3f72f7ffe901b59c,
0x3f627e31fe9ddafb,
0xbf26f641f369bbbd,
0xbf0863f485fc7e89,
0x3ecad77cbbb32bcd,
0x3ea32e705af41964,
0xbe62d9d2bb447e35,
0xbe341f0e4f4dc44a,
0x3df198b51eee9749,
0x3dbec8f104a030ca,
],
[
0x3fd33518b3874e8a,
0xbca7f56ee546a569,
0xbfc33518b3874e3b,
0x3f7d34125d59fb96,
0x3f880c83bdee

In [13]:
# Maclaurin series for j0 
def print_expansion_at_0_f():
    print(f"pub(crate) const J0_MACLAURIN_SERIES: [u64; 9] = [")
    from mpmath import mp, j0, taylor
    mp.prec = 60
    poly = taylor(lambda val: j0(val), 0, 18)
    z = 0
    for i in range(0, 18, 2):
        print(f"{double_to_hex(poly[i])},")
    print("];")

    print(f"poly {poly}")

print_expansion_at_0_f()

pub(crate) const J0_MACLAURIN_SERIES: [u64; 9] = [
0x3ff0000000000000,
0xbfd0000000000000,
0x3f90000000000000,
0xbf3c71c71c71c71c,
0x3edc71c71c71c71c,
0xbe723456789abcdf,
0x3e002e85c0898b71,
0xbd8522a43f65486a,
0x3d0522a43f65486a,
];
poly [mpf('1.0'), mpf('0.0'), mpf('-0.25'), mpf('0.0'), mpf('0.015625'), mpf('0.0'), mpf('-0.00043402777777777777782'), mpf('0.0'), mpf('6.7816840277777777785e-6'), mpf('0.0'), mpf('-6.7816840277777777822e-8'), mpf('0.0'), mpf('4.7095027970679012323e-10'), mpf('0.0'), mpf('-2.4028075495244394055e-12'), mpf('0.0'), mpf('9.3859669903298414278e-15'), mpf('0.0'), mpf('-2.8969033920771115517e-17')]


In [16]:
# Taylor series for f32
mp.prec = 60
print(f"pub(crate) static J0F_COEFFS: [[u64; 15]; {len(j0_zeros)}] = [")

def get_constant_term(poly, y):
    for term in poly.operands():
        if term.is_constant():
            return term

def print_taylor_coeffsf(poly):
    print("[")
    for i in range(0, 15):
        coeff = poly[i]
        print(f"{double_to_hex(coeff)},")
    print("],")

prev_zero = 0

for i in range(0, len(j0_zeros)):
    k_range = j0_zeros[i]
    range_diff = k_range - prev_zero
    g_c = 1

    x0 = mp.mpf(k_range)
    from mpmath import mp, j0, taylor
    poly = taylor(lambda val: j0(val), x0, 15)
    # print(poly)
    print_taylor_coeffsf(poly)
    prev_zero = j0_zeros[i]

print("];")

pub(crate) static J0F_COEFFS: [[u64; 15]; 47] = [
[
0x0000000000000000,
0xbfe09cdb36551280,
0x3fbba1deea029494,
0x3facfae864368d70,
0xbf81bb1cbe1a4071,
0xbf61f992590d12bd,
0x3f315382ba06cc47,
0x3f06ed3b9f07eb28,
0xbed232c77d228ab6,
0xbea1cce302821846,
0x3e67ff99166c20b8,
0x3e32951bd4726a93,
0xbdf5c2c38b2a278c,
0xbdbbdc468c1a817a,
0x3d7cd41cf248a22e,
],
[
0xbfd9c6cf582cbf7f,
0x0000000000000000,
0x3fc9c6cf582cbf7f,
0xbf91f06d14e11e02,
0xbf8b589d1da13905,
0x3f50f9103cf5b152,
0x3f386445621cc085,
0xbefa2a033ccf2705,
0xbed83a06e30c4109,
0x3e96a4fd997104b3,
0x3e6ec03c7b7d1357,
0xbe295db03343bc40,
0xbdfb1e242e3fafb5,
0x3db3fa9bccb27cd1,
0x3d8195cae4f67f9d,
],
[
0x0000000000000000,
0x3fd5c6e60a097823,
0xbf9f8f72e7a848e0,
0xbfab2150cb41e8c1,
0x3f72f7ffe90256bb,
0x3f627e31fe9a9779,
0xbf26f641f41956f7,
0xbf0863f481a43036,
0x3ecad77d748a06db,
0x3ea32e6d99c6af7d,
0xbe62da37e38435b9,
0xbe341d72d9392e0e,
0x3df1d0433d9a0e49,
0x3dbe2f3389aa5f69,
0xbd78498ffdebdd63,
],
[
0x3fd33518b3874e8a,
0x00000000000

In [7]:
# J0 coeffs
mp.prec = 180
def print_taylor_coeffs(poly):
    print("J0TaylorExtendedSeries {")
    print_double_double("a0: ", poly[0])
    print_double_double("a1: ", poly[1])
    print_double_double("a2: ", poly[2])
    print_double_double("a3: ", poly[3])
    print_double_double("a4: ", poly[4])
    print_double_double("a5: ", poly[5])
    print_double_double("a6: ", poly[6])
    print_double_double("a7: ", poly[7])
    print_double_double("a8: ", poly[8])
    print_double_double("a9: ", poly[9])
    print_double_double("a10: ", poly[10])
    print_double_double("a11: ", poly[11])
    print_double_double("a12: ", poly[12])
    print_double_double("a13: ", poly[13])
    print_double_double("a14: ", poly[14])
    print_double_double("a15: ", poly[15])
    print("c: [")
    for i in range(16, 24):
        coeff = poly[i]
        print(f"{double_to_hex(coeff)},")
    print("],")
    print("},")

def print_taylor_coeffs_dyad(poly):
    print("[")
    for i in range(0, 24):
        coeff = poly[i]
        print_dyadic(coeff)
    print("],")

# print(f"pub(crate) static J0_COEFFS: [J0TaylorExtendedSeries; {len(j0_zeros)}] = [")

# prev_zero = 0

# for i in range(0, len(j0_zeros)):
#     k_range = j0_zeros[i]
#     range_diff = k_range - prev_zero

#     mp.prec = 180
#     x0 = mp.mpf(k_range)
#     from mpmath import mp, j0, taylor
#     poly = taylor(lambda val: j0(val), x0, 24)
#     print_taylor_coeffs(poly)
#     # print(poly)
#     prev_zero = j0_zeros[i]

# print("];")

print(f"pub(crate) static J0_COEFFS_RATIONAL128: [[DyadicFloat128; 24]; {len(j0_zeros)}] = [")

prev_zero = 0

for i in range(0, len(j0_zeros)):
    k_range = j0_zeros[i]
    range_diff = k_range - prev_zero

    mp.prec = 180
    x0 = mp.mpf(k_range)
    from mpmath import mp, j0, taylor
    poly = taylor(lambda val: j0(val), x0, 24)
    print_taylor_coeffs_dyad(poly)
    # print(poly)
    prev_zero = j0_zeros[i]

print("];")

pub(crate) static J0_COEFFS_RATIONAL128: [[DyadicFloat128; 24]; 47] = [
[
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -237,
    mantissa: 0xe5d2dbc7_cde683d3_0bf4ff07_98634b4d_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -128,
    mantissa: 0x84e6d9b2_a8940359_1987ad75_f48e1f23_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -131,
    mantissa: 0xdd0ef750_14a49c95_e9dd3e2d_f0ffcc4a_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -132,
    mantissa: 0xe7d74321_b46b8379_3179fb8f_4fd29710_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -134,
    mantissa: 0x8dd8e5f0_d20389a8_7a5518ae_c0c551e7_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -136,
    mantissa: 0x8fcc92c8_6895e7c1_11e20cef_2807ec41_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -139,
    mantissa: 0x8a9c15d0_36623841_4453152c_96aaf5e3_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exp

In [9]:
def compute_intervals(zeros):
    intervals = []
    for i in range(0, len(zeros)):
        if i == 0:
            a = (zeros[i]) / 2 - 0.05 - zeros[i]
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.05 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        elif i + 1 > len(zeros) - 1:
            a = (zeros[i - 1] + zeros[i]) / 2 - 0.05 - zeros[i]
            b = (zeros[i]) + 0.83 + 0.05 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        else:
            a = (zeros[i - 1] + zeros[i]) / 2 - zeros[i] - 0.05
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.05  - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
    return intervals

intervals = compute_intervals(j0_zeros)
# print(intervals)

def build_sollya_script(a, b, zero, deg):
    return f"""
prec = 500;
bessel_j0 = library("/Users/radzivon/RustroverProjects/pxfm/notes/bessel_sollya/cmake-build-release/libbessel_sollya.dylib");
f = bessel_j0(x + {zero});
d = [{a}, {b}];
pf = remez(f, {deg}, d);
for i from 0 to degree(pf) do {{
    write(coeff(pf, i)) >> "coefficients.txt";
    write("\\n") >> "coefficients.txt";
}};
"""

def load_coefficients(filename):
    with open(filename, "r") as f:
        return [RealField(500)(line.strip()) for line in f if line.strip()]

def call_sollya_on_interval(a, b, zero, degree=12):
    sollya_script = build_sollya_script(a, b, zero, degree)
    with open("tmp_interval.sollya", "w") as f:
        f.write(sollya_script)
    import subprocess
    if os.path.exists("coefficients.txt"):
        os.remove("coefficients.txt")
    try:
        result = subprocess.run(
            ["sollya", "tmp_interval.sollya"],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        return

def print_remez_coeffs(poly):
    print("[")
    for i in range(len(poly)):
        coeff = poly[i]
        print_double_double("", coeff)
    print("],")

degree = 23

print(f"pub(crate) static J0_COEFFS: [[(u64, u64); {degree + 1}]; {len(intervals)}] = [")
for i in range(0, len(intervals)):
    interval = intervals[i]
    call_sollya_on_interval(interval[0], interval[1], interval[2], degree)
    coeffs = load_coefficients(f"coefficients.txt")
    print_remez_coeffs(coeffs)
print("];")

];


In [4]:
mp.prec = 180
def print_expansion_at_0():
    print(f"const J0_MACLAURIN_SERIES: [(u64, u64); 12] = [")
    from mpmath import mp, j0, taylor
    poly = taylor(lambda val: j0(val), 0, 24)
    # print(poly)
    real_i = 0
    for i in range(0, 24, 2):
        print_dyadic(DD(poly[i]))
        # print_double_double("", DD(poly[i]))
        real_i = real_i + 1
    print("];")
    print(poly)

print_expansion_at_0()

const J0_MACLAURIN_SERIES: [(u64, u64); 12] = [
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -127,
    mantissa: 0x80000000_00000000_00000000_00000000_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -129,
    mantissa: 0x80000000_00000000_00000000_00000000_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -133,
    mantissa: 0x80000000_00000000_00000000_00000000_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -139,
    mantissa: 0xe38e38e3_8e38e38e_38e38e38_e38e38e4_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -145,
    mantissa: 0xe38e38e3_8e38e38e_38e38e38_e38e38e4_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -151,
    mantissa: 0x91a2b3c4_d5e6f809_1a2b3c4d_5e6f8092_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -158,
    mantissa: 0x81742e04_4c5b8724_8909fcb6_8cd4e410_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -166,
    mantissa: